In [1]:
import numpy as np 
import matplotlib 
import mdtraj as md
import sklearn
from glob import glob
import os 
import pickle as pkl

In [ ]:
#load peptide data
base_data_dir = "/net/data02/nickc/trp-cage/"

embedding_map = {'ALA' : 1,
                'CYS' : 2,
                'ASP' : 3,
                'GLU' : 4,
                'PHE' : 5,
                'GLY' : 6,
                'HIS' : 7,
                'ILE' : 8,
                'LYS' : 9,
                'LEU' : 10,
                'MET' : 11,
                'ASN' : 12,
                'PRO' : 13,
                'GLN' : 14,
                'ARG' : 15,
                'SER' : 16,
                'THR' : 17,
                'VAL' : 18,
                'TRP' : 19,
                'TYR' : 20,
                'N' : 21,
                'CA' : 22,
                'C' : 23,
                'O' : 24}

In [ ]:
def generate_embeddings(residues):
    
    embedding_map = {'ALA' : 1,
                    'CYS' : 2,
                    'ASP' : 3,
                    'GLU' : 4,
                    'PHE' : 5,
                    'GLY' : 6,
                    'HIS' : 7,
                    'ILE' : 8,
                    'LYS' : 9,
                    'LEU' : 10,
                    'MET' : 11,
                    'ASN' : 12,
                    'PRO' : 13,
                    'GLN' : 14,
                    'ARG' : 15,
                    'SER' : 16,
                    'THR' : 17,
                    'VAL' : 18,
                    'TRP' : 19,
                    'TYR' : 20,
                    'N' : 21,
                    'CA' : 22,
                    'C' : 23,
                    'O' : 24}
    
    embeddings = []
    for res in residues:
        identity = embedding_map[res.name]
        if identity == 6:
            residue_embeddings = [21, 6, 23, 24]
        else:
            residue_embeddings = [21, 22, identity, 23, 24]
            
        embeddings += residue_embeddings
    return embeddings

In [ ]:
#peptide meta information
res_lim = 20
meta_dictionary = {}
pdb_file = base_data_dir + 'trpcage_50ns_0/structure.pdb'
pdb = md.load_pdb(pdb_file)   
    
#resides
residues = []
for resnum, res in enumerate(pdb.topology.residues):
    if resnum < res_lim:
        residues.append(res)
    else:
        break
glys = [name for name in residues if name.name == 'GLY']
num_gly = len(glys)

gly_idx = [i for i, res in enumerate(residues) if res.name == 'GLY']

#print(residues)

#get all the atoms within the residues
num_atoms = 0
for res in residues:
    num_atoms += len(list(res.atoms))

#atoms

# BEWARE - FOR PROLINE  O N L Y
# THE ORDER OF CB and CA ARE S W I T C H E D
# Later, we manually re-switch them!


atoms_needed = ['N','CA', 'CB', 'C', 'O']
atom_indices = []
atoms_grabbed = []
md_atoms = []
for idx, atom in enumerate(pdb.topology.atoms):
    residue = atom.residue
    atom_name = atom.name
    if residue in residues:
        if atom_name in atoms_needed:
            #print(residue, atom_name, idx )
            atom_indices.append(idx)
            atoms_grabbed.append(atom_name)
            md_atoms.append(atom)

assert len(atom_indices) <= 100

#assemble dictionary

gly_c_idx = [j for j, name in enumerate(md_atoms) if str(name)[:3] == 'GLY' and name.name[-1] == 'C']  

meta_dictionary['residues'] = residues
meta_dictionary['num_gly'] = num_gly
meta_dictionary['gly_idx'] = gly_idx
meta_dictionary['gly_c_idx'] = gly_c_idx
meta_dictionary['cg_atoms'] = md_atoms 
meta_dictionary['cg_mapping_per_residue'] = atoms_needed
meta_dictionary['cg_indices'] = atom_indices
meta_dictionary['num_all_atoms'] = num_atoms

resseq = []

for residue in residues:
    if residue.name == 'GLY':
        for _ in range(4):
            resseq.append(6)
    else:
        for _ in range(5):
            resseq.append(embedding_map[residue.name])         

meta_dictionary['resmap'] = {k:v.name for k,v in zip(np.arange(1, res_lim + 1), residues)}
embeddings = generate_embeddings(residues)
meta_dictionary['resseq'] = resseq
assert len(embeddings) == len(atom_indices)
meta_dictionary['embeddings']  = embeddings

gly_shift = []
current_shift = 0
for residue in residues:
    if residue.name == 'GLY':
        gly_shift.append(np.array([current_shift, current_shift, current_shift + 1, current_shift + 1, current_shift + 1]))   
        current_shift += 1
    else:
        gly_shift.append(np.array([current_shift for _ in range(5)]))
gly_shift = np.concatenate(gly_shift).flatten()
#print(gly_shift)

meta_dictionary['gly_shift'] = gly_shift
#if num == 144:
#    print(meta_dictionary)
#print(meta_dictionary[0])
pkl.dump(meta_dictionary, open('trpcage_meta_dictionary_no_physical_GLY.pkl', 'wb'))

In [ ]:
for k, v in meta_dictionary.items():
    print(k,v)

In [ ]:
# Here we assemble ONLY the coordinate data

coord_base_dir = 'coords_nowater/'

coord_list = []
coord_files = sorted(glob(base_data_dir + 'coords_nowater/*.npy'))
#assert len(coord_files) == len(force_files)

#load the files, checking against the meta
j=0
for coord_file in coord_files:
    print(coord_file)
    coord = np.load(coord_file)
    #force = np.load(force_file)
    #assert coord.shape == force.shape
    assert coord.shape[1] == meta_dictionary['num_all_atoms']
    cg_coord = coord[:, meta_dictionary['cg_indices'], :]
    #cg_force = force[:, meta_dictionary['cg_indices'], :]
    # Here, we swap the CA and CB order for PROLINES ONLY:
    # First, sniff for prolines:
    proline_residue_idx = []
    for idx, res in meta_dictionary['resmap'].items():
        if res == "PRO":
            proline_residue_idx.append(idx - 1)
    if len(proline_residue_idx) > 0:
        print("prolines detected at: ",proline_residue_idx)
        # for each proline, we swap the rows of the CB and CA
        for res_idx in proline_residue_idx:
            #CB is located at 1 in PDB order, and CA is at 2
            #we need CB at 2 and CA at 1
            #Find out how many GLY precede this PRO 
            #CB is located at 1 in PDB order, and CA is at 2
            #we need CB at 2 and CA at 1
            unshifted_cb_idx = (res_idx*5) + 1
            unshifted_ca_idx = (res_idx*5) + 2
            old_cb_idx = unshifted_cb_idx - gly_shift[unshifted_cb_idx]
            old_ca_idx = unshifted_ca_idx - gly_shift[unshifted_ca_idx]
            #print(old_cb_idx)
            #print(old_ca_idx)
            #print('old')
            #print(cg_coord[0,[old_cb_idx, old_ca_idx],:])
            #print(cg_force[0,[old_cb_idx, old_ca_idx],:])
            cb_coords = np.copy(cg_coord[:, old_cb_idx, :])
            ca_coords = np.copy(cg_coord[:, old_ca_idx, :])
            #print(cb_coords[0])

            #cb_forces = np.copy(cg_force[:, old_cb_idx, :])
            #ca_forces = np.copy(cg_force[:, old_ca_idx, :])

            cg_coord[:, old_cb_idx, :] = ca_coords
            cg_coord[:, old_ca_idx, :] = cb_coords
            #print(cb_coords[0])
            #cg_force[:, old_cb_idx, :] = ca_forces
            #cg_force[:, old_ca_idx, :] = cb_forces
            #print('new')
            #print(cg_coord[0,[old_cb_idx, old_ca_idx],:])
            #print(cg_force[0,[old_cb_idx, old_ca_idx],:])

    assert cg_coord.shape[1] <= 195  # should be the same for all peptides (5 beads per residue, 8 residues)
    coord_list.append(cg_coord)
    np.save('/net/data02/nickc/trp-cage/coords_nowater/cg_mapped/trpcage_opep_coord_no_physical_GLY_{:04}.npy'.format(j),
           cg_coord) 
    j += 1
    #force_list.append(cg_force)

#assert len(coord_list) == len(force_list)

cg_coords = np.concatenate(coord_list, axis=0)
#cg_forces = np.concatenate(force_list, axis=0)
np.save('/net/data02/nickc/trp-cage/coords_nowater/cg_mapped/opep_mapping_trpcage_coords_no_physical_GLY.npy', cg_coords)
#np.save(base_numpy_data_dir + 'brooke_map_cg_data/' + 'opep_{:04d}_cg_forces.npy'.format(num), cg_forces)

In [ ]:
print(cg_coords.shape)
